In [ ]:
# imports
from datasets import load_dataset
import json

In [ ]:
# Dataset preparation
dataset = load_dataset("spider")
training = dataset["train"]

In [ ]:
#Schemas

with open("tables.json", "r") as f:
    tables_json = json.load(f)

schemas = {}
for db in tables_json:
    db_id = db["db_id"]
    schemas[db_id] = {
        "tables": db["table_names_original"],  
        "columns": db["column_names_original"], 
        "column_types": db["column_types"],     
        "foreign_keys": db["foreign_keys"],     
        "primary_keys": db["primary_keys"],     
    }



In [ ]:
def formatDatabaseSchema(schema):
  tables = schema["tables"]
  columns = schema["columns"]
  col_types = schema["column_types"]

In [ ]:
inputs = []
targets = []

for example in training :
  db_id = example["db_id"]
  schemaText = formatDatabaseSchema[schemas[db_id]]
  question = example["question"]
  sqlQuery = example["query"]

  inputs.append(f"Question: {question} \n {schemaText}")
  targets.append(sqlQuery)
